In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Download historical data for desired stock
symbol = "VOW.DE"
data = yf.download(symbol, start='2000-01-01', end='2023-12-02')

# Initial values for the parameters
initial_n_days = 20
initial_up_target = data['Adj Close'].iloc[-1] + 50
initial_down_target = data['Adj Close'].iloc[-1] - 50

def estimate_probability(n_days, initial_price, up_target, down_target):
    # Calculate the thresholds for price change
    up_threshold = round(abs((up_target - initial_price) / initial_price), 3)
    down_threshold = round(abs((down_target - initial_price) / initial_price), 3)

    # Calculate the n-day percentage change
    data[f'{n_days}d_pct_change'] = data['Adj Close'].pct_change(n_days)

    # Calculate the frequency of the threshold price change
    total_periods = len(data)
    down_periods = len(data[data[f'{n_days}d_pct_change'] <= -down_threshold])
    up_periods = len(data[data[f'{n_days}d_pct_change'] >= up_threshold])
    down_frequency = down_periods / total_periods
    up_frequency = up_periods / total_periods

    # Plotting
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(24, 8))
    sns.histplot(data=data, x=f'{n_days}d_pct_change', kde=True, bins=100, ax=ax1)
    ax1.axvline(x=-down_threshold, color='r', linestyle='--', label=f"{down_threshold * 100:.2f}% decrease frequency: {down_frequency * 100:.2f}%")
    ax1.axvline(x=up_threshold, color='b', linestyle='--', label=f"{up_threshold * 100:.2f}% increase frequency: {up_frequency * 100:.2f}%")

    ax1.set_xlabel('Percentage change')
    ax1.set_ylabel('Frequency')
    ax1.set_title(f'Distribution of {n_days}-day percentage changes for {symbol}')
    ax1.legend(fontsize=20)

    data['Adj Close'].plot(ax=ax2)
    up_instances = data[data[f'{n_days}d_pct_change'] >= up_threshold]
    down_instances = data[data[f'{n_days}d_pct_change'] <= -down_threshold]
    ax2.scatter(up_instances.index, up_instances['Adj Close'], color='b', marker='^', label=f"{up_threshold * 100:.2f}% increase")
    ax2.scatter(down_instances.index, down_instances['Adj Close'], color='r', marker='v', label=f"{down_threshold * 100:.2f}% decrease")

    ax2.set_xlabel('Date')
    ax2.set_ylabel('Adjusted Close Price')
    ax2.set_title(f'Stock Price for {symbol}')
    ax2.legend(fontsize=20)

    plt.tight_layout()
    plt.show()

# Creating sliders for interactive inputs with initialized values
n_days_slider = widgets.IntSlider(min=1, max=100, step=1, value=initial_n_days, description="n_days")
initial_price_slider = widgets.FloatSlider(min=1, max=1000, step=0.01, value=data['Adj Close'].iloc[-1], description="Initial Price")
up_target_slider = widgets.FloatSlider(min=1, max=1000, step=0.01, value=initial_up_target, description="Up Target")
down_target_slider = widgets.FloatSlider(min=1, max=1000, step=0.01, value=initial_down_target, description="Down Target")

# Use interact function with estimate_probability function and the sliders
interact(estimate_probability, n_days=n_days_slider, initial_price=initial_price_slider, up_target=up_target_slider, down_target=down_target_slider);

[*********************100%%**********************]  1 of 1 completed


interactive(children=(IntSlider(value=20, description='n_days', min=1), FloatSlider(value=124.6500015258789, d…